In [64]:
from copyreg import constructor
from evogym import is_connected, has_actuator, get_full_connectivity, hashable
from evogym.envs import StepsUp, WalkingFlat
import struct
from typing import List, Tuple
import numpy as np
import dill
from graph import Graph
from evolution_strategies import tournament_selection
from population import Population
from arg_parser import parse_args
import imageio
import test

In [65]:
FILE_PATH = "../ladder/"
GENERATION = "20"
ENV = "StepsUp" # "[WalkingFlat, StepsUp]"

In [66]:
with open(f"{FILE_PATH}structure_{GENERATION}.pkl", "rb") as file:
    structure_pop = dill.load(file)
    
with open(f"{FILE_PATH}controler_{GENERATION}.pkl", "rb") as file:
    controller_pop = dill.load(file)
    
if ENV == "StepsUp":
    def get_observation(env):
        a = env.get_vel_com_obs("robot")
        b = env.get_pos_com_obs("robot")
        c = env.get_floor_obs("robot", ["ground"], 5)
        return np.concatenate((a, b, c))
else:
    def get_observation(env):
        a = env.get_vel_com_obs("robot")
        b = env.get_pos_com_obs("robot")
        return np.concatenate((a, b))
    
def generate_robot(g: Graph, structure):
    robot = np.zeros(structure)
    for i in range(structure[0]):
        for j in range(structure[1]):
            input = (i - (structure[0] // 2),
                     j - (structure[1] // 2))
            graph_out = g.operate(input)
            node = np.argmax(graph_out)
            robot[i][j] = node
    return robot

def get_best_controller(robot, controller_population, n_steps):
    robot = generate_robot(design, (5,5))    
    connections = get_full_connectivity(robot)

    if ENV == "StepsUp":
        env = StepsUp(body=robot, connections=connections)
    else:
        env = WalkingFlat(body=robot, connections=connections)
    env.reset()
    
    fitness = []
    cont = 0
    for controller in controller_population.indvs:
        reward = test.calculate_reward(env, controller, 500)
        fitness.append((cont, reward))
        cont += 1
        
    fitness.sort(key=lambda x: (x[1], x[0]))
    
    return fitness

In [67]:
structure_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
controller_pop.indvs.sort(key=lambda x: (x.original_fit*-1, x.id))
cont = 0
print("\t ID \t  Fitness \t Species")
for design in structure_pop.indvs:
    if design.original_fit > -999:
       
        print(f"{cont}:\t {design.id}\t| {design.original_fit:.4f}\t| {design.species_id}")
    cont+=1

	 ID 	  Fitness 	 Species
0:	 2128	| 1.6732	| 68
1:	 1486	| 1.5386	| 68
2:	 1750	| 1.4584	| 68
3:	 2120	| 1.2718	| 68
4:	 2114	| 0.6774	| 68
5:	 2098	| 0.2092	| 68
6:	 2116	| 0.1517	| 6
7:	 2106	| 0.0667	| 68
8:	 2129	| 0.0574	| 68
9:	 2118	| 0.0393	| 68
10:	 2096	| -3.0050	| 68


In [68]:
STEPS = 500
CONTROLLER_ID = 1
DESIGN_ID = 2
SAVE_GIF = False

In [69]:
controller = controller_pop.indvs[CONTROLLER_ID]
design =structure_pop.indvs[DESIGN_ID]

robot = generate_robot(design, (5,5))
print(robot)
connections = get_full_connectivity(robot)

if ENV == "StepsUp":
    env = StepsUp(body=robot, connections=connections)
else:
    env = WalkingFlat(body=robot, connections=connections)



env.reset()
env.render('screen')

reward = 0
actuators = env.get_actuator_indices("robot")
imgs = []
for _ in range(STEPS):
    obs = get_observation(env)

    action_by_actuator = controller.operate(obs, reset_fit=False)
    action = [action_by_actuator[i] for i in actuators]
#     action = np.clip(action, .6, 1.6)
    _, r, done, _ = env.step(np.array(action))
    
    if SAVE_GIF:
        imgs.append(env.render(mode='img'))
    env.render('screen')
    reward += r

    if done:
        print("AAA")
        break

        
print(f'\ntotal reward: {round(reward, 5)}\n')
env.close()

if SAVE_GIF:
    imageio.mimsave(f'{ENV}_gen_{GENERATION}_c{CONTROLLER_ID}_d_{DESIGN_ID}.gif', imgs, duration=(1/50.0))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 4. 4.]
 [2. 1. 1. 4. 4.]
 [0. 1. 1. 4. 4.]
 [0. 0. 1. 0. 0.]]

total reward: 1.55681



In [70]:
aux = get_best_controller(robot, controller_pop, 200)
print(aux)

[(11, -0.09125906127527794), (12, -0.0422769195774062), (18, -0.027374097071538372), (2, -0.016964866013839286), (6, -0.014535163024595121), (9, -0.011476202742538577), (19, -0.010781859724958043), (16, -0.009436795662332687), (1, -0.0073647259861706615), (7, -0.004573211805759314), (14, -0.003088256936597311), (4, -0.0027644429019275663), (5, 0.004542883966621725), (17, 0.010327394217517272), (10, 0.017754633641395623), (3, 0.017856830848064664), (8, 0.01792297134755083), (13, 0.03594248603409689), (15, 0.05373580451433124), (0, 1.458393179463169)]
